In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import random
import pickle
from sklearn.metrics import confusion_matrix, f1_score, roc_curve ,accuracy_score, precision_score, recall_score , classification_report
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Embedding, LSTM, Dropout, Dense ,GRU,Flatten,Lambda,Concatenate
from keras.models import load_model
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical
from keras import backend as K
from keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import ModelCheckpoint, CSVLogger, History,EarlyStopping
from keras.layers import GlobalAveragePooling1D,TimeDistributed,SpatialDropout1D,Embedding,GlobalMaxPooling1D, Reshape, Activation, Input, Dense, Masking, Conv1D, MaxPooling1D, Bidirectional


def load(name):
    try:
        with open(name, "rb") as f:
            data = pickle.load(f)
    except Exception as e:
        print(e)
        data = []
    return data

def save(obj,name):
    with open(name, "wb") as f:
        pickle.dump(obj, f, protocol=2)
    return

Using TensorFlow backend.


In [3]:
# How to map char to index (some number)
CHARS = ['N','A','T','C','G']
CHARS_MAP = {v: k for k, v in enumerate(CHARS)}
char = np.array(CHARS)
print(CHARS_MAP)

{'N': 0, 'A': 1, 'T': 2, 'C': 3, 'G': 4}


In [4]:
####################################################################################################################

In [5]:
CHARS_tri= ['aaa','aat','aac','aag','ata','att','atc','atg','aca','act','acc','acg','aga','agt','agc','agg','taa','tat','tac','tag','tta','ttt','ttc','ttg','tca','tct','tcc','tcg','tga','tgt','tgc','tgg','caa','cat','cac','cag','cta','ctt','ctc','ctg','cca','cct','ccc','ccg','cga','cgt','cgc','cgg','gaa','gat','gac','gag','gta','gtt','gtc','gtg','gca','gct','gcc','gcg','gga','ggt','ggc','ggg']
CHARS_tri_map = {v: k for k, v in enumerate(CHARS_tri,1)}
print(CHARS_tri_map)

{'aaa': 1, 'aat': 2, 'aac': 3, 'aag': 4, 'ata': 5, 'att': 6, 'atc': 7, 'atg': 8, 'aca': 9, 'act': 10, 'acc': 11, 'acg': 12, 'aga': 13, 'agt': 14, 'agc': 15, 'agg': 16, 'taa': 17, 'tat': 18, 'tac': 19, 'tag': 20, 'tta': 21, 'ttt': 22, 'ttc': 23, 'ttg': 24, 'tca': 25, 'tct': 26, 'tcc': 27, 'tcg': 28, 'tga': 29, 'tgt': 30, 'tgc': 31, 'tgg': 32, 'caa': 33, 'cat': 34, 'cac': 35, 'cag': 36, 'cta': 37, 'ctt': 38, 'ctc': 39, 'ctg': 40, 'cca': 41, 'cct': 42, 'ccc': 43, 'ccg': 44, 'cga': 45, 'cgt': 46, 'cgc': 47, 'cgg': 48, 'gaa': 49, 'gat': 50, 'gac': 51, 'gag': 52, 'gta': 53, 'gtt': 54, 'gtc': 55, 'gtg': 56, 'gca': 57, 'gct': 58, 'gcc': 59, 'gcg': 60, 'gga': 61, 'ggt': 62, 'ggc': 63, 'ggg': 64}


In [6]:
CHARS_bi= ['aa','at','ac','ag','tt','ta','tc','tg','cc','ca','ct','cg','gg','ga','gt','gc']
CHARS_bi_map = {v: k for k, v in enumerate(CHARS_bi,1)}
print(CHARS_bi_map)

{'aa': 1, 'at': 2, 'ac': 3, 'ag': 4, 'tt': 5, 'ta': 6, 'tc': 7, 'tg': 8, 'cc': 9, 'ca': 10, 'ct': 11, 'cg': 12, 'gg': 13, 'ga': 14, 'gt': 15, 'gc': 16}


In [7]:
dict_neg = {"data":[]}
dict_train = {"data":[]}
dict_all_neg = {"data":[]}

classTrain_1_in = open('/Users/pariwattongnussook/Desktop/thesis/TAE-ML/Data_used/YH_GATK_101_train_test_dup.txt','r')
i=0
for line in classTrain_1_in:
    separator = '\t'
    line = line.replace('\n','') 
    line = line.split(separator)
    STRING = line[2]
    STRING_list = []
    ############################################
    if line[26]=='2' or line[26]=='1':
        ref_a=0
        ref_t=0
        ref_c=0
        ref_g=0
        ref_n=0
        #
        alt_a=0
        alt_t=0
        alt_c=0
        alt_g=0
        alt_n=0
        #
        i_a=0
        i_t=0
        i_c=0
        i_g=0
        i_n=0
        #
        aa=0
        at=0
        ac=0
        ag=0
        tt=0
        ta=0
        tc=0
        tg=0
        cc=0
        ca=0
        ct=0
        cg=0
        gg=0
        ga=0
        gt=0
        gc=0
        for idex in range(len(STRING)-1):
            #print(STRING[idex],STRING[idex+1])
            item = STRING[idex]+STRING[idex+1]
            if item.lower() =="aa":
                aa+=1
            elif item.lower() =="at":
                at+=1
            elif item.lower() =="ac":
                ac+=1
            elif item.lower() =="ag":
                ag+=1     
            elif item.lower() =="tt":
                tt+=1
            elif item.lower() =="ta":
                ta+=1
            elif item.lower() =="tc":
                tc+=1  
            elif item.lower() =="tg":
                tg+=1
            elif item.lower() =="cc":
                cc+=1
            elif item.lower() =="ca":
                ca+=1  
            elif item.lower() =="ct":
                ct+=1
            elif item.lower() =="cg":
                cg+=1
            elif item.lower() =="gg":
                gg+=1  
            elif item.lower() =="ga":
                ga+=1
            elif item.lower() =="gt":
                gt+=1  
            elif item.lower() =="gc":
                gc+=1
        ############################################
        #BI
        item_l =0
        item_r =0
        item_l_bi = STRING[50-2]+STRING[50-1]
        if item_l_bi.lower() in CHARS_bi_map:
            item_l = CHARS_bi_map[item_l_bi.lower()]
        ############################################
        item_r_bi = STRING[50+1]+STRING[50+2]
        if item_r_bi.lower() in CHARS_bi_map:
            item_r = CHARS_bi_map[item_r_bi.lower()]
        ############################################
        #TRI
        item2_l =0
        item2_r =0
        item_l_tri = STRING[50-3]+STRING[50-2]+STRING[50-1]
        if item_l_tri.lower() in CHARS_tri_map:
            item2_l = CHARS_tri_map[item_l_tri.lower()]
        ############################################
        item_r_tri = STRING[50+1]+STRING[50+2]+STRING[50+3]
        if item_r_tri.lower() in CHARS_tri_map:
            item2_r = CHARS_tri_map[item_r_tri.lower()]
        ############################################
        ref = CHARS_MAP[line[3]]
        alt = CHARS_MAP[line[4]]
        #sub = ALU_map[line[7]]
        sub = 0
        most = 0
        if line[3]+line[4] == 'AG' or line[3]+line[4] == 'TC':
            most =1
        bi = CHARS_bi_map[line[3].lower()+line[4].lower()] 
        dict_train["data"].append({"REF": ref, "ALT": alt,"bi":bi,"MOST":most,"BI_L":item_l,"BI_R":item_r,"TRI_L":item2_l,"TRI_R":item2_r,"QUAL":line[5],"Subfamilies":sub, "AC": line[8], "AF": line[9], "AN": line[10], "DP": line[11], "FS": line[12], "MQ": line[13], "QD": line[14], "SOR": line[15], "DS": line[16], "EXC": line[17], "HPS": line[18], "IBC": line[19], "MAC": line[20], "MAF": line[21], "MQR": line[22], "RPS": line[23],"DUP_QTY": line[24],"DUP_ALL": line[25],"aa":aa,"at":at,"ac":ac,"ag":ag,"tt":tt,"ta":ta,"tc":tc,"tg":tg,"cc":cc,"ca":ca,"ct":ct,"cg":cg,"gg":gg,"ga":ga,"gt":gt,"gc":gc,"class":1})


In [8]:
i_pos = len(dict_train["data"])
print("Train Positive:",len(dict_train["data"]))

Train Positive: 62624


In [9]:
# Negative Train Dataset
classTrain_0_in = open('/Users/pariwattongnussook/Desktop/thesis/TAE-ML/Data_used/YH_GATK_101_train_test_dup.txt','r')
classTrain_0 = []
STRING = ''
i=0
for line in classTrain_0_in:
    separator = '\t'
    line = line.replace('\n','') 
    line = line.split(separator)
    STRING = line[2]
    STRING_list = []
    ############################################
    if line[26]=='0':
        ref_a=0
        ref_t=0
        ref_c=0
        ref_g=0
        ref_n=0
        #
        alt_a=0
        alt_t=0
        alt_c=0
        alt_g=0
        alt_n=0
        #
        i_a=0
        i_t=0
        i_c=0
        i_g=0
        i_n=0
        #
        aa=0
        at=0
        ac=0
        ag=0
        tt=0
        ta=0
        tc=0
        tg=0
        cc=0
        ca=0
        ct=0
        cg=0
        gg=0
        ga=0
        gt=0
        gc=0
        for idex in range(len(STRING)-1):
            #print(STRING[idex],STRING[idex+1])
            item = STRING[idex]+STRING[idex+1]
            if item.lower() =="aa":
                aa+=1
            elif item.lower() =="at":
                at+=1
            elif item.lower() =="ac":
                ac+=1
            elif item.lower() =="ag":
                ag+=1     
            elif item.lower() =="tt":
                tt+=1
            elif item.lower() =="ta":
                ta+=1
            elif item.lower() =="tc":
                tc+=1  
            elif item.lower() =="tg":
                tg+=1
            elif item.lower() =="cc":
                cc+=1
            elif item.lower() =="ca":
                ca+=1  
            elif item.lower() =="ct":
                ct+=1
            elif item.lower() =="cg":
                cg+=1
            elif item.lower() =="gg":
                gg+=1  
            elif item.lower() =="ga":
                ga+=1
            elif item.lower() =="gt":
                gt+=1  
            elif item.lower() =="gc":
                gc+=1
        ############################################
        #BI
        item_l =0
        item_r =0
        item_l_bi = STRING[50-2]+STRING[50-1]
        if item_l_bi.lower() in CHARS_bi_map:
            item_l = CHARS_bi_map[item_l_bi.lower()]
        ############################################
        item_r_bi = STRING[50+1]+STRING[50+2]
        if item_r_bi.lower() in CHARS_bi_map:
            item_r = CHARS_bi_map[item_r_bi.lower()]
        ############################################
        #TRI
        item2_l =0
        item2_r =0
        item_l_tri = STRING[50-3]+STRING[50-2]+STRING[50-1]
        if item_l_tri.lower() in CHARS_tri_map:
            item2_l = CHARS_tri_map[item_l_tri.lower()]
        ############################################
        item_r_tri = STRING[50+1]+STRING[50+2]+STRING[50+3]
        if item_r_tri.lower() in CHARS_tri_map:
            item2_r = CHARS_tri_map[item_r_tri.lower()]
        ############################################
        ref = CHARS_MAP[line[3]]
        alt = CHARS_MAP[line[4]]
        #sub = ALU_map[line[7]]
        sub = 0
        most = 0
        if line[3]+line[4] == 'AG' or line[3]+line[4] == 'TC':
            most =1
        bi = CHARS_bi_map[line[3].lower()+line[4].lower()] 
        dict_neg["data"].append({"REF": ref, "ALT": alt,"bi":bi,"MOST":most,"BI_L":item_l,"BI_R":item_r,"TRI_L":item2_l,"TRI_R":item2_r,"QUAL":line[5],"Subfamilies":sub, "AC": line[8], "AF": line[9], "AN": line[10], "DP": line[11], "FS": line[12], "MQ": line[13], "QD": line[14], "SOR": line[15], "DS": line[16], "EXC": line[17], "HPS": line[18], "IBC": line[19], "MAC": line[20], "MAF": line[21], "MQR": line[22], "RPS": line[23],"DUP_QTY": line[24],"DUP_ALL": line[25],"aa":aa,"at":at,"ac":ac,"ag":ag,"tt":tt,"ta":ta,"tc":tc,"tg":tg,"cc":cc,"ca":ca,"ct":ct,"cg":cg,"gg":gg,"ga":ga,"gt":gt,"gc":gc,"class":0})


In [10]:
import random
random.shuffle(dict_neg["data"])
i=0
for val in dict_neg["data"]:
    if i < i_pos:
        dict_train["data"].append(val)
    else:
        dict_all_neg["data"].append(val)
    i+=1

In [11]:
print("All train Pos,Neg:",len(dict_train["data"]))

All train Pos,Neg: 125248


In [12]:
print("All the rest neg:",len(dict_all_neg["data"]))

All the rest neg: 71663


In [13]:
import random
random.shuffle(dict_train["data"])

In [14]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [15]:
df_train = pd.DataFrame(dict_train["data"],columns=["REF","ALT","MOST","BI_L","BI_R","TRI_L","TRI_R","QUAL","AC","AF","AN","DP","FS","MQ","SOR","DUP_QTY",'aa','at','ac','ag','tt','ta','tc','tg','cc','ca','ct','cg','gg','ga','gt','gc','class'])
df_train = df_train.apply(pd.to_numeric)
df_train.head(100)


,REF,ALT,MOST,BI_L,BI_R,TRI_L,TRI_R,QUAL,AC,AF,AN,DP,FS,MQ,SOR,DUP_QTY,aa,at,ac,ag,tt,ta,tc,tg,cc,ca,ct,cg,gg,ga,gt,gc,class
0,2,3,1,3,1,19,1,73.77,1,0.5,2,6,3.979,50.0,1.414,0,13,5,5,7,2,7,5,7,9,7,6,3,6,3,8,7,1
1,1,4,1,15,5,30,22,35.77,1,0.5,2,8,0.000,50.0,0.368,0,3,8,6,7,12,6,5,7,5,9,5,3,7,6,6,5,0
2,1,4,1,11,7,26,26,47.74,2,1.0,2,2,0.000,50.0,2.303,0,11,6,5,7,6,5,10,8,3,7,11,1,4,6,5,5,0
3,1,4,1,10,14,25,49,46.77,1,0.5,2,5,0.000,50.0,1.609,1,5,3,4,7,2,2,11,6,11,8,11,5,7,3,5,10,1
4,3,2,0,9,15,43,55,21.77,2,1.0,2,2,0.000,50.0,2.303,0,12,7,6,6,1,4,5,7,8,8,5,4,10,7,4,6,0
5,1,4,1,5,10,6,36,82.77,1,0.5,2,8,0.000,50.0,0.818,1,1,6,3,9,10,6,7,6,10,7,8,3,6,5,5,8,1
6,4,1,0,3,16,35,58,103.77,1,0.5,2,6,0.000,50.0,1.609,0,5,1,8,8,4,2,6,9,9,10,11,1,8,4,5,9,0
7,1,4,1,16,13,15,62,51.28,2,1.0,2,3,0.000,50.0,2.833,0,3,5,4,11,5,2,4,10,7,12,5,0,12,6,6,8,0
8,2,3,1,7,10,39,36,40.81,1,0.5,2,4,0.000,50.0,1.609,0,4,4,4,9,2,3,13,6,8,11,11,2,6,4,6,7,1
9,4,1,0,8,15,40,53,62.74,2,1.0,2,2,0.000,50.0,2.303,0,11,3,9,6,13,8,4,6,4,6,7,2,7,3,8,3,0


In [16]:
df_train.to_csv(r'/Users/pariwattongnussook/Desktop/thesis/TAE-ML/Train_YH/TrainData.csv')

In [17]:
array = df_train.values

In [18]:
dict_temp = {"data":[]}
# Test Dataset
classTrain_0_in = open('/Users/pariwattongnussook/Desktop/thesis/TAE-ML/Data_used/Hela_GATK_101_train_test_nofilter_dup.txt','r')
classTrain_0 = []
STRING = ''
i=0
for line in classTrain_0_in:
    separator = '\t'
    line = line.replace('\n','') 
    line = line.split(separator)
    STRING = line[2]
    STRING_list = []
    ############################################
    if line[26]=='0' or line[26]=='1':
        ref_a=0
        ref_t=0
        ref_c=0
        ref_g=0
        ref_n=0
        #
        alt_a=0
        alt_t=0
        alt_c=0
        alt_g=0
        alt_n=0
        #
        i_a=0
        i_t=0
        i_c=0
        i_g=0
        i_n=0
        #
        aa=0
        at=0
        ac=0
        ag=0
        tt=0
        ta=0
        tc=0
        tg=0
        cc=0
        ca=0
        ct=0
        cg=0
        gg=0
        ga=0
        gt=0
        gc=0
        for idex in range(len(STRING)-1):
            #print(STRING[idex],STRING[idex+1])
            item = STRING[idex]+STRING[idex+1]
            if item.lower() =="aa":
                aa+=1
            elif item.lower() =="at":
                at+=1
            elif item.lower() =="ac":
                ac+=1
            elif item.lower() =="ag":
                ag+=1     
            elif item.lower() =="tt":
                tt+=1
            elif item.lower() =="ta":
                ta+=1
            elif item.lower() =="tc":
                tc+=1  
            elif item.lower() =="tg":
                tg+=1
            elif item.lower() =="cc":
                cc+=1
            elif item.lower() =="ca":
                ca+=1  
            elif item.lower() =="ct":
                ct+=1
            elif item.lower() =="cg":
                cg+=1
            elif item.lower() =="gg":
                gg+=1  
            elif item.lower() =="ga":
                ga+=1
            elif item.lower() =="gt":
                gt+=1  
            elif item.lower() =="gc":
                gc+=1
        ############################################
        #BI
        item_l =0
        item_r =0
        item_l_bi = STRING[50-2]+STRING[50-1]
        if item_l_bi.lower() in CHARS_bi_map:
            item_l = CHARS_bi_map[item_l_bi.lower()]
        ############################################
        item_r_bi = STRING[50+1]+STRING[50+2]
        if item_r_bi.lower() in CHARS_bi_map:
            item_r = CHARS_bi_map[item_r_bi.lower()]
        ############################################
        #TRI
        item2_l =0
        item2_r =0
        item_l_tri = STRING[50-3]+STRING[50-2]+STRING[50-1]
        if item_l_tri.lower() in CHARS_tri_map:
            item2_l = CHARS_tri_map[item_l_tri.lower()]
        ############################################
        item_r_tri = STRING[50+1]+STRING[50+2]+STRING[50+3]
        if item_r_tri.lower() in CHARS_tri_map:
            item2_r = CHARS_tri_map[item_r_tri.lower()]
        ############################################
        ref = CHARS_MAP[line[3]]
        alt = CHARS_MAP[line[4]]
        #sub = ALU_map[line[7]]
        sub = 0
        most = 0
        if line[3]+line[4] == 'AG' or line[3]+line[4] == 'TC':
            most =1
        bi = CHARS_bi_map[line[3].lower()+line[4].lower()] 
        dict_temp["data"].append({"REF": ref, "ALT": alt,"bi":bi,"MOST":most,"BI_L":item_l,"BI_R":item_r,"TRI_L":item2_l,"TRI_R":item2_r,"QUAL":line[5],"Subfamilies":sub, "AC": line[8], "AF": line[9], "AN": line[10], "DP": line[11], "FS": line[12], "MQ": line[13], "QD": line[14], "SOR": line[15], "DS": line[16], "EXC": line[17], "HPS": line[18], "IBC": line[19], "MAC": line[20], "MAF": line[21], "MQR": line[22], "RPS": line[23],"DUP_QTY": line[24],"DUP_ALL": line[25],"aa":aa,"at":at,"ac":ac,"ag":ag,"tt":tt,"ta":ta,"tc":tc,"tg":tg,"cc":cc,"ca":ca,"ct":ct,"cg":cg,"gg":gg,"ga":ga,"gt":gt,"gc":gc,"class":line[26]})


In [19]:
ipos = len(dict_temp["data"])
print("Test positive:",ipos)

Test positive: 236552


In [20]:
random.shuffle(dict_temp["data"])
print("Test pos,neg:",len(dict_temp["data"]))

Test pos,neg: 236552


In [21]:
df_test = pd.DataFrame(dict_temp["data"],columns=["REF","ALT","MOST","BI_L","BI_R","TRI_L","TRI_R","QUAL","AC","AF","AN","DP","FS","MQ","SOR","DUP_QTY",'aa','at','ac','ag','tt','ta','tc','tg','cc','ca','ct','cg','gg','ga','gt','gc','class'])
df_test = df_test.apply(pd.to_numeric)
df_test.head(100)


,REF,ALT,MOST,BI_L,BI_R,TRI_L,TRI_R,QUAL,AC,AF,AN,DP,FS,MQ,SOR,DUP_QTY,aa,at,ac,ag,tt,ta,tc,tg,cc,ca,ct,cg,gg,ga,gt,gc,class
0,2,3,1,7,3,23,10,148.77,1,0.5,2,11,7.404,50.0,3.611,0,14,6,6,6,5,7,3,8,7,8,5,1,8,3,7,6,1
1,4,1,0,8,9,40,41,102.28,2,1.0,2,3,0.000,50.0,1.179,0,11,4,4,12,3,4,5,5,9,9,5,1,9,7,5,7,0
2,1,4,1,3,9,35,42,399.77,2,1.0,2,6,0.000,50.0,0.693,0,13,4,8,7,1,6,8,5,5,8,9,3,8,4,6,5,0
3,3,1,0,6,5,21,22,21.77,2,1.0,2,2,0.000,50.0,0.693,0,17,10,4,9,13,11,3,4,2,4,4,2,2,9,3,3,0
4,1,4,1,9,10,59,34,48.74,2,1.0,2,2,0.000,50.0,0.693,0,23,9,5,7,8,5,3,8,2,7,3,1,3,8,4,4,0
5,4,1,0,2,5,18,23,91.28,2,1.0,2,3,0.000,50.0,1.179,0,9,7,3,11,5,6,5,10,2,5,3,1,12,10,10,1,0
6,3,4,0,6,3,5,9,114.90,2,1.0,2,5,0.000,50.0,1.022,0,13,9,9,8,6,6,5,5,5,15,2,1,2,5,6,3,0
7,4,3,0,5,16,54,58,398.77,2,1.0,2,11,0.000,50.0,0.859,0,0,2,7,3,5,1,10,12,11,10,14,1,8,1,7,8,0
8,3,2,0,6,10,5,36,21.77,2,1.0,2,2,0.000,50.0,0.693,0,10,8,8,9,5,9,6,7,2,6,10,0,4,9,4,3,0
9,2,3,1,10,8,9,31,217.94,1,0.5,2,7,0.000,50.0,0.527,0,10,11,5,4,15,10,7,6,5,7,7,0,3,2,6,2,0


In [22]:
array_test = df_test.values
test_real_x = array_test[:,0:((df_test.values.shape[1])-1)]
test_real_y = array_test[:,((df_test.values.shape[1])-1)]

In [23]:
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
# scikit-learn k-fold cross-validation
from numpy import array
from sklearn.model_selection import KFold

In [24]:
# Random Forest Classification
kfold = KFold(5)
# enumerate splits
i=1
for train, test in kfold.split(df_train.values):
    print("\n")
    print('K:',i)
    i+=1
    max_length = ((df_train.values[train].shape[1])-1)
    train_x = df_train.values[train][:,0:max_length]
    train_y = df_train.values[train][:,max_length]
    test_x = df_train.values[test][:,0:max_length]
    test_y = df_train.values[test][:,max_length]
    #max_features = ['log2']
    max_features = ['auto', 'sqrt']
    num_trees = 600
    model_RF = RandomForestClassifier(n_estimators=num_trees, max_features=max_features[0],criterion="entropy",min_samples_leaf=5)
    trained_RF = model_RF.fit(train_x, train_y)
    print ("Trained model :: ", trained_RF)
    predictions = trained_RF.predict(test_x)
    ###############################################################################################################################   
    print("Confusion matrix\n")
    print(pd.crosstab(pd.Series(test_y, name='Actual'), pd.Series(predictions, name='Predicted')))
    def get_metrics(y_test, y_predicted):
        accuracy = accuracy_score(y_test, y_predicted)
        precision = precision_score(y_test, y_predicted, average='weighted')
        recall = recall_score(y_test, y_predicted, average='weighted')
        f1 = f1_score(y_test, y_predicted, average='weighted')
        return accuracy, precision, recall, f1
    accuracy, precision, recall, f1 = get_metrics(test_y, predictions)
    print("accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall, f1))




K: 1
Trained model ::  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=600,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Confusion matrix

Predicted   0.0    1.0
Actual                
0.0        9640   2930
1.0        1221  11259
accuracy = 0.834 
precision = 0.841 
recall = 0.834 
f1 = 0.834


K: 2
Trained model ::  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
          

In [25]:
# Random Forest Classification
max_length = ((df_train.values.shape[1])-1)
train_data = df_train.values[:int(len(df_train.values)*0.6)]
test_data = df_train.values[int(len(df_train.values)*0.6):int(len(df_train.values)*0.8)]
train_x = train_data[:,0:max_length]
train_y = train_data[:,max_length]
test_x = test_data[:,0:max_length]
test_y = test_data[:,max_length]
#max_features = ['log2']
max_features = ['auto', 'sqrt']
num_trees = 600
model_RF = RandomForestClassifier(n_estimators=num_trees, max_features=max_features[0],criterion="entropy",min_samples_leaf=5)
trained_RF = model_RF.fit(train_x, train_y)
print ("Trained model :: ", trained_RF)
predictions = trained_RF.predict(test_x)
###############################################################################################################################   
print("Confusion matrix\n")
print(pd.crosstab(pd.Series(test_y, name='Actual'), pd.Series(predictions, name='Predicted')))
def get_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average='weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 = f1_score(y_test, y_predicted, average='weighted')
    return accuracy, precision, recall, f1
accuracy, precision, recall, f1 = get_metrics(test_y, predictions)
print("accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall, f1))


Trained model ::  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=600,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Confusion matrix

Predicted   0.0    1.0
Actual                
0.0        9575   2939
1.0        1249  11287
accuracy = 0.833 
precision = 0.839 
recall = 0.833 
f1 = 0.832


In [26]:
y_pred = trained_RF.predict(test_real_x)
###############################################################################################################################
print("Confusion matrix\n")
print(pd.crosstab(pd.Series(test_real_y, name='Actual'), pd.Series(y_pred, name='Predicted')))
def get_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average='weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 = f1_score(y_test, y_predicted, average='weighted')
    return accuracy, precision, recall, f1
accuracy, precision, recall, f1 = get_metrics(test_real_y, y_pred)
print("accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall, f1))

Confusion matrix

Predicted     0.0    1.0
Actual                  
0.0        196522  28680
1.0          2087   9263
accuracy = 0.870 
precision = 0.954 
recall = 0.870 
f1 = 0.901
